# Unary Iteration

Given an array of potential operations, for example:

    ops = [X(i) for i in range(5)]
    
we would like to select an operation to apply:

    n = 4 --> apply ops[4]
    
If $n$ is a quantum integer, we need to apply the transformation

$$
    |n \rangle |\psi\rangle \rightarrow |n\rangle \, \mathrm{ops}_n \cdot |\psi\rangle
$$

The simplest conceptual way to do this is to use a "total control" quantum circuit where you introduce a multi-controlled operation for each of the `len(ops)` possible `n` values.

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit
import numpy as np
from typing import *

In [ ]:
import operator
import functools
import itertools

## Total Control

Here, we'll use Sympy's boolean logic to show how total control works. We perform an `And( ... )` for each possible bit pattern. We use an `Xnor` on each selection bit to toggle whether it's a positive or negative control (filled or open circle in quantum circuit diagrams).

In this example, we indeed consider $X_n$ as our potential operations and toggle bits in the `target` register according to the total control.

In [ ]:
import sympy as S
import sympy.logic.boolalg as slb

def total_control(selection, target):
    """Toggle bits in `target` depending on `selection`."""
    print(f"Selection is {selection}")
    
    for n, trial in enumerate(itertools.product((0, 1), repeat=len(selection))):
        print(f"Step {n}, apply total control: {trial}")
        target[n] ^= slb.And(*[slb.Xnor(s, t) for s, t in zip(selection, trial)])
          
        if target[n] == S.true:
            print(f"  -> At this stage, {n}= and our output bit is set")

        
selection = [0, 1, 0]
target = [False]*8
total_control(selection, target)    
print()
print("Target:")
print(target)

Note that our target register shows we have indeed applied $X_\mathrm{0b010}$. Try changing `selection` to other bit patterns and notice how it changes.

Of course, we don't know what state the selection register will be in. We can use sympy's support for symbolic boolean logic to verify our gadget for all possible selection inputs.

In [ ]:
selection = [S.Symbol(f's{i}') for i in range(3)]
target = [S.false for i in range(2**len(selection)) ]
total_control(selection, target)

print()
print("Target:")
for n, t in enumerate(target):
    print(f'{n}= {t}')
    
tc_target = target.copy()

As expected, the "not pattern" (where `~` is boolean not) matches the binary representations of `n`.

## Unary Iteration with segment trees

A [segment tree](https://en.wikipedia.org/wiki/Segment_tree) is a data structure thatallows logrithmic-time querying of intervals. We use a segment tree where each interval is length 1 and comprises all the `n` integers we may select.

It is defined recursively by dividing the input interval into two half-size intervals until the left limit meets the right limit.

In [ ]:
def segtree(ctrl, selection, target, depth, left, right):
    """Toggle bits in `target` depending on `selection` using a recursive segment tree."""
    print(f'{depth=} {left=} {right=}', end=' ')
    
    if left == (right - 1):
        # Leaf of the recusion.
        print(f'{n=} {ctrl=}')
        target[left] ^= ctrl
        return 
    print()
    
    assert depth < len(selection)
    mid = (left + right) >> 1
    
    # Recurse left interval
    new_ctrl = slb.And(ctrl, slb.Not(selection[depth]))
    segtree(ctrl=new_ctrl, selection=selection, target=target, depth=depth+1, left=left, right=mid)
    
    # Recurse right interval
    new_ctrl = slb.And(ctrl, selection[depth])
    segtree(ctrl=new_ctrl, selection=selection, target=target, depth=depth+1, left=mid, right=right)
    
    # Quantum note:
    # instead of throwing away the first value of `new_ctrl` and re-anding
    # with selection, we can just invert the first one (but only if `ctrl` is active)
    # new_ctrl ^= ctrl

In [ ]:
selection = [S.Symbol(f's{i}') for i in range(3)]
target = [S.false for i in range(2**len(selection)) ]
segtree(S.true, selection, target, 0, 0, 2**len(selection))

print()
print("Target:")
for n, t in enumerate(target):
    print(f'{n=} {slb.simplify_logic(t)}')

In [ ]:
print(f"{'n':3s} | {'segtree':18s} | {'total control':18s} | same?")
for n, (t1, t2) in enumerate(zip(target, tc_target)):
    t1 = slb.simplify_logic(t1)
    print(f'{n:3d} | {str(t1):18s} | {str(t2):18s} | {str(t1==t2)}')

## Quantum Circuit

We can translate the boolean logic to reversible, quantum logic.

In [ ]:
from cirq_qubitization.cirq_infra.gate_with_registers import Registers
from cirq_qubitization.cirq_algos.unary_iteration import UnaryIterationGate
from functools import cached_property

# TODO: mpharrigan thinks that UnaryIterationGate should have-a operation rather
# than be subclassed each time.

class ApplyXToLthQubit(UnaryIterationGate):
    def __init__(self, selection_bitsize: int, target_bitsize: int, control_bitsize: int = 1):
        self._selection_bitsize = selection_bitsize
        self._target_bitsize = target_bitsize
        self._control_bitsize = control_bitsize

    @cached_property
    def control_registers(self) -> Registers:
        return Registers.build(control=self._control_bitsize)

    @cached_property
    def selection_registers(self) -> Registers:
        return Registers.build(selection=self._selection_bitsize)

    @cached_property
    def target_registers(self) -> Registers:
        return Registers.build(target=self._target_bitsize)

    @cached_property
    def iteration_lengths(self) -> Tuple[int, ...]:
        return (self._target_bitsize,)

    def nth_operation(
        self, control: cirq.Qid, selection: int, target: Sequence[cirq.Qid]
    ) -> cirq.OP_TREE:
        return cirq.CNOT(control, target[-(selection + 1)])

In [ ]:
import cirq_qubitization.cirq_infra.testing as cq_testing

selection_bitsize = 3
target_bitsize = 5
g = cq_testing.GateHelper(
    ApplyXToLthQubit(selection_bitsize, target_bitsize))
SVGCircuit(cirq.Circuit(cirq.decompose_once(g.operation)))

## Tests for Correctness

We can use a full statevector simulation to compare the desired statevector to the one generated by the unary iteration circuit for each basis state.

In [ ]:
from cirq_qubitization.bit_tools import iter_bits

for n in range(target_bitsize):
    # Initial qubit values
    qubit_vals = {q: 0 for q in g.all_qubits}
    # All controls 'on' to activate circuit
    qubit_vals |= {c: 1 for c in g.quregs['control']}
    # Set selection according to `n`
    qubit_vals |= zip(g.quregs['selection'], iter_bits(n, selection_bitsize))

    initial_state = [qubit_vals[x] for x in g.all_qubits]
    qubit_vals[g.quregs['target'][-(n + 1)]] = 1
    final_state = [qubit_vals[x] for x in g.all_qubits]
    cq_testing.assert_circuit_inp_out_cirqsim(
        g.decomposed_circuit, g.all_qubits, initial_state, final_state
    )
    print(f'{n=} checked!')